In [1]:
import pandas as pd
from translate import Translator
import re
import json
# Ler arquivo utils para entender de onde os dados são buscados
from utils import FetchCID

In [2]:
df = pd.read_excel('src/SimpleTabulation-ICD-11-MMS-pt.xlsx')

In [3]:
df = df[['Code', 'Title', 'TitleEN', 'Foundation URI']]

In [4]:
df.head()

,Code,Title,TitleEN,Foundation URI
0,NaN,Algumas doenças infecciosas ou parasitárias,Certain infectious or parasitic diseases,http://id.who.int/icd/entity/1435254666
1,NaN,- Gastroenterites ou colites de origem infecciosa,- Gastroenteritis or colitis of infectious origin,http://id.who.int/icd/entity/588616678
2,NaN,- - Infecções intestinais bacterianas,- - Bacterial intestinal infections,http://id.who.int/icd/entity/135352227
3,1A00,- - - Cólera,- - - Cholera,http://id.who.int/icd/entity/257068234
4,1A01,- - - Infecção intestinal por outras bactérias...,- - - Intestinal infection due to other Vibrio,http://id.who.int/icd/entity/416025325


### Verificado que vários items não possuiam tradução para o português, iremos traduzir o texto da coluna TitleEN

In [5]:
translator= Translator(to_lang="pt-BR")
def translate_title(item):
    # Sentença que verifica valores NaN
    if item.Title != item.Title:
        item.Title = translator.translate(item.TitleEN)
    return item

df = df.apply(translate_title, axis=1)

### Funções auxiliares

In [6]:
def delete_hyphens(text):
    return re.sub(r"^[-\s]+", "", text).strip()

# Evitar contar hifens entre as palavras
def count_leading_hyphens(text):
    match = re.match(r"^([- ]+)", text)
    
    if match:
        return match.group(0).count('-')
    return 0

### Criado coluna "level"
Nesta coluna será armazenado a hierarquia dos códigos  

**EX:**  
Algumas doenças infecciosas ou parasitárias - Level 1  
    Gastroenterites ou colites de origem infecciosa - Level 1.1 (É o primeiro elemento filho do elemento pai 1)  
    Infecções intestinais bacterianas 	- Level 1.2 (É o segundo elemento filho do elemento pai 1)  

In [7]:
hyphens = 0
current_level = []
current_leaf = 0

def level(title):
    global hyphens, current_level, current_leaf
    if count_leading_hyphens(title) == hyphens:
        if hyphens == 0:
            current_level.append(current_leaf)
        current_leaf += 1
        current_level[-1] = current_leaf
        return '.'.join([str(x) for x in current_level])
    if count_leading_hyphens(title) > hyphens:
        hyphens = count_leading_hyphens(title)
        current_leaf = 1
        current_level.append(current_leaf)
        return '.'.join([str(x) for x in current_level])
    if count_leading_hyphens(title) < hyphens:
        aux = count_leading_hyphens(title) - hyphens
        current_level = current_level[:aux]
        current_level[-1] += 1
        current_leaf = current_level[-1]
        hyphens = count_leading_hyphens(title)
        return '.'.join([str(x) for x in current_level])

df['level'] = df['Title'].apply(level)


### Criando dicionário com hierarquia aplicada

Cada elemento pai é uma chave e seus filhos são subchaves do mesmo, assim sucessivamente

In [8]:
icd_dict = {}
title_level_map = dict(zip(df['level'], df['Title']))
code_level_map = dict(zip(df['level'], df['Code']))

fetch_cid = FetchCID()

def create_dict(item):
    list_level = item.level.split('.')
    str_levels = []
    for l in range(len(list_level)-1):
        str_levels.append('.'.join(list_level[:-(l+1)]))
        
    if len(list_level) == 1:
        key_name = item.Title
        icd_dict.update({key_name: {
            "cid": item.Code,
            "description": fetch_cid.get_description(item['Foundation URI']),
            "subitens": {}
        }})
    if len(list_level) > 1:
        last_node = icd_dict
        for node in str_levels[::-1]:
            node_name = delete_hyphens(title_level_map[node])
            last_node = last_node[node_name]['subitens']

        key_name = delete_hyphens(item.Title)
        last_node.update({key_name: {
            "cid": item.Code,
            "description": fetch_cid.get_description(item['Foundation URI']),
            "subitens": {}
        }})      

df.apply(create_dict, axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
36039    None
36040    None
36041    None
36042    None
36043    None
Length: 36044, dtype: object

### Transformando dicionário em arquivo .json

In [9]:
str_data = json.dumps(icd_dict, ensure_ascii=False)
j = json.loads(str_data)
with open('data.json', 'w', encoding='utf-8') as file:
    file.write(str_data)